# 通过api获取，failed

In [ ]:
import requests
import json
import pandas as pd

In [4]:
def rgb_to_hex(r, g, b):
    return "#{:02x}{:02x}{:02x}".format(r, g, b)

def fetch_color_info(hex_color):
    url = "https://www.qtccolor.com/webapi/color/Search"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json, text/plain, */*",
        "X-Requested-With": "XMLHttpRequest"
    }
    data = {
        "hex": hex_color,
        'brand': '',
    }
    
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

In [ ]:
if __name__ == "__main__":
    # 示例RGB值
    
    
    
    # 获取颜色信息
    color_info = fetch_color_info(hex_color)
    
    if color_info:
        print("获取到的颜色信息：")
        print(json.dumps(color_info, indent=4))
    else:
        print("未能获取颜色信息。")

# 通过request、xpath获取网页信息

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyperclip
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from lxml import etree
import re
import os

In [11]:
r, g, b = 224, 141, 123


In [72]:
def rgb_to_hex(r, g, b):
    return "#{:02x}{:02x}{:02x}".format(r, g, b)
def fetch_color_info(r,g,b):
    hex_color = rgb_to_hex(r, g, b)[1:]
    url = f'https://www.qtccolor.com/secaiku/search?hex={hex_color}&brand='
    response = requests.get(url)
    root = etree.HTML(response.text)
    ColorBrand = root.xpath("//div[@id='findColor']//div[@class='ColorBrand']")
    ColorCode = root.xpath("//div[@id='findColor']//div[@class='ColorCode']")
    delta = root.xpath("//i[@class='delta']")
    # colorData = root.xpath("//div[@class='el-col el-col-24 el-col-xs-8 el-col-sm-4']")
    hex = root.xpath("//img[@class='larger-color-image pointer']/@style")
    ColorList = []
    for i in range(len(delta)):
        ColorList.append([ColorBrand[i].text,ColorCode[i].text,delta[i].text,hex[i][11:-2]])
    return ColorList

In [73]:
fetch_color_info(r,g,b)

[['PANTONE', '4051 CP', 'ΔE 1.6', '#DD897'],
 ['PANTONE', '16-1329TPG', 'ΔE 2.4', '#E3928'],
 ['PANTONE', '4052 C', 'ΔE 2.4', '#DA887'],
 [None, 'SM1-0552', 'ΔE 2.5', '#E7948'],
 ['RAL', '040 70 40', 'ΔE 2.7', '#E8917'],
 ['PANTONE', '486 C', 'ΔE 3.2', '#E8927'],
 ['PANTONE', '486 UP', 'ΔE 3.2', '#E88A7'],
 ['PANTONE', 'P 55-3 C', 'ΔE 3.4', '#E4948'],
 ['PANTONE', '4052 U', 'ΔE 3.9', '#DD8B8'],
 ['PANTONE', '15-1520TCX', 'ΔE 3.9', '#EB968'],
 ['PANTONE', 'P 57-12 U', 'ΔE 4', '#DB887'],
 ['PANTONE', '15-1520TPG', 'ΔE 4', '#EC978'],
 ['CBCC', '0223 10R7/7.2', 'ΔE 4.2', '#EA927'],
 ['PANTONE', '486 CP', 'ΔE 4.4', '#E4937'],
 ['PANTONE', 'P 57-2 U', 'ΔE 4.4', '#EA978'],
 ['PANTONE', 'P 52-4 U', 'ΔE 4.8', '#EC998'],
 ['PANTONE', 'P 40-12 U', 'ΔE 4.9', '#D5846'],
 ['PANTONE', '17-1341TCX', 'ΔE 5.1', '#D37F6'],
 ['RAL', '430-2', 'ΔE 5.1', '#EC998'],
 ['PANTONE', '16-1526TCX', 'ΔE 5.1', '#D3837'],
 ['PANTONE', '4053 U', 'ΔE 5.2', '#D3827'],
 ['PANTONE', '177 CP', 'ΔE 5.2', '#E37F7'],
 ['PANTON

# Autocad

In [84]:
# Set autocad environment and set an empty list to store material name
from pyautocad import Autocad
acad = Autocad(create_if_not_exists=True)

In [85]:
# Define the function to extract material from the drawing
def extract_text_from_dwg(dwg_file):
    try:    
        acad = Autocad(create_if_not_exists=True)
        doc = acad.app.Documents.Open(dwg_file)  
        time.sleep(1)    
        for i in range(doc.Layouts.Count):
            layout = doc.Layouts[i]
            print('extracting from layout: %s' % layout.Name)
            for entity in acad.iter_objects(['AcDbText','AcDbMText','AcDbMLeader','TDbSymbMultiLeader','TDbSymbIndexPointer','TDbSymbComposing','TDbText'],block=layout.Block,dont_cast=True):
                name = entity.ObjectName
                if name == 'AcDbText' or name == 'AcDbMText' or name == 'AcDbMLeader':
                    extracted_text.append(entity.TextString)
                if name == 'TDbSymbMultiLeader':
                    extracted_text.append([entity.UpText,entity.DownText])
                if name == 'TDbSymbIndexPointer':
                    extracted_text.append([entity.UpText,entity.DownText]) #上标文字，下标文字
                if name == 'TDbSymbComposing'or name == 'TDbText':
                    extracted_text.append(entity.Text) #文字内容
        doc.Close()
        return extracted_text        
    except Exception as e:
        return str(e)


In [86]:
# Define the function to find all dwg files in the folder
def find_dwg_files(file_path):
    dwg_files = []
    for root, dirs, files in os.walk(file_path):
        for file in files:
            if file.endswith(".dwg"):
                dwg_files.append(os.path.join(root, file))
    return dwg_files

In [87]:
# Set the path to the folder where the dwg files are located,and find all the dwg files in the folder
file_path = r'W:\城建院数据\20220526 西藏项目\05-施工图纸\中心公园\广场'
dwg_files = find_dwg_files(file_path)
#Empty list to store extracted text
extracted_text = []
# Extract all text from dwg files in a folder
for dwg_file in dwg_files:
    try:
        t1 = time.time()
        print('Opening file: ' + dwg_file)
        extract_text_from_dwg(dwg_file)
        t2 = time.time()
        print(f'Time taken: ' + str(t2-t1) + ' seconds')
    except:
        print('Error in file: ' + dwg_file)
        pass
extracted_text

Opening file: W:\城建院数据\20220526 西藏项目\05-施工图纸\中心公园\广场\YS-01建筑附属广场.dwg
Time taken: 8.780218839645386 seconds
Opening file: W:\城建院数据\20220526 西藏项目\05-施工图纸\中心公园\广场\YS-020304礼仪广场、音乐广场、数学广场-wkf.dwg
extracting from layout: Model
Time taken: 147.2991647720337 seconds
Opening file: W:\城建院数据\20220526 西藏项目\05-施工图纸\中心公园\广场\YS-0506健身广场、书法广场-wsd.dwg
Time taken: 0.15412664413452148 seconds
Opening file: W:\城建院数据\20220526 西藏项目\05-施工图纸\中心公园\广场\YS-0708体育广场、下沉广场-bxn.dwg
Time taken: 1.108246088027954 seconds
Opening file: W:\城建院数据\20220526 西藏项目\05-施工图纸\中心公园\广场\YS-09舞动广场.dwg
Time taken: 0.008019208908081055 seconds
Opening file: W:\城建院数据\20220526 西藏项目\05-施工图纸\中心公园\广场\YS-10水磨台.dwg
Time taken: 0.42551612854003906 seconds


[['浅蓝色彩色塑胶地垫', '(RGB值：135,206,235)'],
 ['中黄色彩色塑胶地垫', '(RGB值：255,215,0)'],
 ['500X150X200白麻烧面花岗岩平缘石', ''],
 ['500X500X30厚芝麻灰烧面花岗岩座面', ''],
 ['%%c500~1000X100厚圆形青石板', '粒径20~30白色卵石填缝，1:3水泥砂浆粘牢'],
 ['5厚钢板收边', ''],
 ['5厚钢板收边', ''],
 '绿地',
 '绿地',
 '绿地',
 '绿地',
 '铺装',
 '铺装',
 '铺装',
 '铺装',
 '坐凳',
 '绿地',
 '绿地',
 '绿地',
 '绿地',
 '绿地',
 '绿地',
 '绿地',
 '绿地',
 '绿地',
 '绿地',
 '坐凳',
 '坐凳',
 '铺装',
 '铺装',
 '铺装',
 '铺装',
 '铺装',
 '铺装',
 '铺装',
 '铺装',
 '铺装',
 '铺装',
 '铺装',
 '铺装',
 ['白色涂料喷涂音符图案', ''],
 ['白色涂料喷涂音符图案', ''],
 ['浅蓝色彩色塑胶地垫', '(RGB值：135,206,235)'],
 ['中黄色彩色塑胶地垫', '(RGB值：255,215,0)'],
 ['500X150X200白麻烧面花岗岩平缘石', ''],
 ['500X150X200白麻烧面花岗岩平缘石', ''],
 ['500X150X200白麻烧面花岗岩平缘石', ''],
 ['500X500X30厚花岗岩坐凳面', ''],
 ['500X500X30厚花岗岩坐凳面', ''],
 ['%%c500~1000X100厚圆形青石板', '粒径20~30白色卵石填缝，1:3水泥砂浆粘牢'],
 ['%%c500~1000X100厚圆形青石板', '粒径20~30白色卵石填缝，1:3水泥砂浆粘牢'],
 ['%%c500~1000X100厚圆形青石板', '粒径20~30白色卵石填缝，1:3水泥砂浆粘牢'],
 ['砖红色彩色塑胶地垫', ''],
 ['砖红色彩色塑胶地垫', '(RGB值：178,34,34)'],
 ['中黄色彩色塑胶地垫', ''],
 ['白色涂料喷涂音符图案', ''],
 ['砖红色仿石涂料',

In [78]:
extrancted_text

[]